In [4]:
from imports import *

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
modelfile = r'C:\Users\f.gionnane\Documents\RAGs\Ollama Learning\Modelfile'

In [ ]:
models_list = ollama.list().model_dump_json()
models_list

In [ ]:
# poser la question "what can you tell me about this image ?" et utiliser llava:7b  
# sur l'image suivante: ./images.jpeg

text = "قله دماوند، بلندترین قله آتشفشانی ایران است که در ۶۵ کیلومتری شمال تهران قرار دارد. این قله با ۵۶۱۰ متر ارتفاع به عنوان بام ایران شناخته می‌شود و مناظر دیدنی آن علاقه‌مندان زیادی را در فصول مختلف سال به خصوص بهار و تابستان به این قله‌ی مرتفع می‌کشاند. دمای قله دماوند در تابستان‌ها صفر درجه بوده و در زمستان‌ها حدود ۶۰ درجه زیر صفر است. شکوه و عظمت این جاذبه‌ی طبیعی بی‌نظیر باعث شده قله دماوند در فرهنگ و تمدن ایران اهمیت ویژه‌ای داشته باشد و به وفور می‌توان ردپای این کوه را در شعرها، افسانه‌ها و داستان‌های پارسی مشاهده کرد."


response = ollama.chat(
    model='llama3.2',
    messages=[
        {'role': 'user',
        'content': f"can you tell me in english what does this text talks about ?\n\n {text}"   
         } ])
reply = response.message.content
reply

Streaming

In [ ]:
# poser la question "what can you tell me about this image ?" et utiliser llava:7b  
# sur l'image suivante: ./images.jpeg

text_ru = "700 миллионов лет назад лед стал угрозой для всех сложных форм жизни на нашей планете, и едва не сделал ее необитаемой. Некоторые ученые считают, что планета была полностью покрыта льдом, ее так и называют – Земля-снежок. Казалось бы, немыслимая судьба для планеты, находящейся так близко к Солнцу. Но в результате этой климатической катастрофы возникли новые формы жизни, которые в итоге привели к появлению современного живого мира."

response = ollama.chat(
    model='llama3.2',
    messages= [
        {'role': 'user',
        'content': f"can you tell me in english what does this text talks about ?\n\n {text_ru}"   
         } ],
         stream=True)
for chunk in response:
    print(chunk.message.content, end='', flush= True)

In [ ]:
# show
llama3_2_description_dict = dict(ollama.show('llama3.2'))
# show keys
llama3_2_description_dict.keys()

In [ ]:
# ouvrir le modelfile
with open(modelfile,'r') as file:
    content = file.read()

content

# Write the modelfile content to a temporary file
with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix='.modelfile') as f:
    f.write(content)
    modelfile_path = f.name

# Use subprocess to run the ollama command with proper encoding handling
try:
    # Force UTF-8 encoding for both stdout and stderr
    result = subprocess.run(
        ['ollama', 'create', 'persian_expert', '-f', modelfile_path],
        capture_output=True,
        text=True,
        encoding='utf-8',  # Explicitly set encoding to UTF-8
        errors='replace',  # Replace any characters that can't be decoded
        check=True
    )
    print("Success:", result.stdout)
except subprocess.CalledProcessError as e:
    print("Error:", e.stderr)
except UnicodeDecodeError as e:
    print(f"Unicode error: {e}")

# Clean up the temporary file
try:
    os.unlink(modelfile_path)
except Exception as e:
    print(f"Cleanup error: {e}")

In [ ]:
ollama.list().model_dump_json()

Grocery List

In [ ]:
input_file = r'grocery_list.txt'
output_file = r'grocery_list.txt'
if not os.path.exists(input_file):
    print(f"{input_file} not found !")
    exit(1)

with open(input_file,'r') as file:
    items = file.read().strip()

prompt = f"""
I have provided you with a grocery list. For each item in the list, I want you to perform the following tasks:

1. Sort the items by main categories such as "Fruits", "Vegetables", "Dairy Products", "Meat", "Drinks", "Cleaning Products", etc.
2. If there are items that can be grouped under a more specific subcategory, do so. For example, group all types of bread under "Bread and Bakery Products".
3. Sort the subcategories alphabetically or by type of product, depending on what makes the most sense.

The list of grocery items you will be processing is as follows:
{items}

Once sorted, provide the list in a structured, easy-to-read format with each category followed by its subcategories and items.
"""



Etude de la librairie MarkDown

In [ ]:
# Download the page and extract the text content
urls = ["https://www.exito.com/portatil-asus-intel-core-i5-1235u-ram-16gb-ssd-1tb-pantalla-156-fhd-quiet-blue-color-103275870-mp/p"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()
print(f"HTML Loaded: {docs}")

# Convert the documents to markdown format
md = MarkdownifyTransformer()

# Assuming 'md' is a markdown processing tool that transforms documents
converted_docs = md.transform_documents(docs)

# Extract the text content and clean it
text_content = converted_docs[0].page_content

# Advanced cleaning process
# Remove images and links (markdown style)
cleaned_text = re.sub(r'!\[.*?\]\(.*?\)', '', text_content)  # Remove images
cleaned_text = re.sub(r'\[.*?\]\(.*?\)', '', cleaned_text)  # Remove markdown links

# Remove extra newlines
cleaned_text = re.sub(r'\n+', '\n', cleaned_text)  # Remove extra newlines
cleaned_text = cleaned_text.strip()  # Remove leading and trailing spaces

# Remove headers, if needed, to focus on content only
cleaned_text = re.sub(r'#{1,6} .*', '', cleaned_text)  # Remove markdown headers

# Remove any code blocks (useful if you want to focus on text content only)
cleaned_text = re.sub(r'```.*?```', '', cleaned_text, flags=re.DOTALL)  # Remove code blocks

# Remove any inline code (if applicable)
cleaned_text = re.sub(r'`.*?`', '', cleaned_text)  # Remove inline code

# Remove any blockquotes if present
cleaned_text = re.sub(r'>.*\n', '', cleaned_text)  # Remove blockquotes

# Remove any potential footnotes or citations
cleaned_text = re.sub(r'\[\^.*?\]', '', cleaned_text)  # Remove footnotes

# Remove any table structures or inline tables
cleaned_text = re.sub(r'\|.*?\|', '', cleaned_text)  # Remove inline tables

# Print the first 1000 characters of the cleaned text for debugging
print(cleaned_text[:1000])


Trying PDF RAGs

In [13]:
DATA_PATH = "PDF"

In [18]:
def load_documents(DATA_PATH: str):
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()

documents = load_documents(DATA_PATH)
documents[0]

Document(metadata={'producer': 'Acrobat Distiller 4.0 for Windows', 'creator': 'PyPDF', 'creationdate': 'D:20010322180048', 'moddate': '2003-11-10T10:46:42-03:00', 'title': 'García Márquez - Cien años de soledad', 'subject': 'Novela', 'author': 'Patricio', 'source': 'PDF\\Cien Anos de Soledad - Gabriel Garcia Marquez.pdf', 'total_pages': 173, 'page': 0, 'page_label': '1'}, page_content='Gabriel García Márquez \nCien años de soledad')

In [22]:
# afficher toutes les pages
for i, doc in enumerate(documents):
    print(f"Page {i+1} of {len(documents)}:")
    print(doc.page_content[:1000])
    print("\n"+ "+" + "-"*50 + "\n")

Page 1 of 173:
Gabriel García Márquez 
Cien años de soledad

+--------------------------------------------------

Page 2 of 173:
Para Jomi García Ascot 
y María Luisa Elio

+--------------------------------------------------

Page 3 of 173:
Cien años de soledad 
Gabriel  García Márquez 
 3 
 
I 
 
Muchos años después, frente al pelotón de fusilamiento, el coronel Aureliano Buendía había de 
recordar aquella tarde remota en que su padre lo llevó a conocer el hielo. Macondo era entonces 
una aldea de veinte casas de barro y cañabrava construidas a la orilla de un río de aguas diáfanas 
que se precipitaban por un lecho de piedras pulidas, blancas y enormes como huevos 
prehistóricos. El mundo era tan reciente, que muchas cosas carecían de nombre, y para 
mencionarlas había que señalarías con el dedo. Todos los años, por el mes de marzo, una familia 
de gitanos desarrapados plantaba su carpa cerca de la aldea, y con un grande alboroto de pitos y 
timbales daban a conocer los nuevos invento